In [2]:
import numpy as np
import pandas as pd
from PIL import Image
import os

In [3]:
images_list = []
dir = 'images/monet_jpg'
for image in os.listdir(dir):
    images_list.append(np.asarray(Image.open(dir +'/' + image)))
images = pd.Series(images_list)
images

0      [[[180, 178, 139], [193, 191, 153], [202, 202,...
1      [[[15, 36, 17], [66, 87, 70], [93, 111, 97], [...
2      [[[178, 196, 206], [190, 208, 218], [199, 217,...
3      [[[160, 164, 191], [163, 169, 195], [164, 170,...
4      [[[154, 139, 118], [163, 148, 127], [163, 148,...
                             ...                        
295    [[[105, 112, 156], [99, 106, 152], [95, 103, 1...
296    [[[238, 248, 240], [236, 246, 238], [239, 249,...
297    [[[130, 139, 154], [75, 87, 99], [53, 66, 72],...
298    [[[219, 223, 186], [219, 223, 186], [220, 222,...
299    [[[249, 249, 247], [247, 247, 245], [244, 244,...
Length: 300, dtype: object

In [123]:
Image.fromarray(images[0]).show()

In [49]:
Image.fromarray(np.rot90(images[0], 3)).show()

In [68]:
#Image.fromarray(np.flip(np.flip(images[0],0), 1)).show()
#Image.fromarray(np.flip(images[0],2)).show()
Image.fromarray(np.rot90(np.flip(np.flip(images[0],0), 1), 3)).show()

In [54]:
flipped_image_list = []
for image in images_list:
    flipped_image_list.append(np.flip(image, 0))
    flipped_image_list.append(np.flip(image, 1))
    flipped_image_list.append(np.flip(np.flip(image, 0), 1))
    flipped_image_list.append(np.rot90(image))
    flipped_image_list.append(np.rot90(image, 3))
    flipped_image_list.append(np.flip(np.rot90(image), 0))
    flipped_image_list.append(np.flip(np.rot90(image), 1))
    flipped_image_list.append(np.flip(image, -1))
    flipped_image_list.append(np.flip(np.flip(image, 0), -1))
    flipped_image_list.append(np.flip(np.flip(image, 1), -1))
    flipped_image_list.append(np.flip(np.flip(np.flip(image, 0), 1), -1))
    flipped_image_list.append(np.flip(np.rot90(image), -1))
    flipped_image_list.append(np.flip(np.rot90(image, 3), -1))
    flipped_image_list.append(np.flip(np.flip(np.rot90(image), 0), -1))
    flipped_image_list.append(np.flip(np.flip(np.rot90(image), 1), -1))
flipped_images = pd.Series(flipped_image_list)
flipped_images

0       [[[147, 127, 118], [185, 167, 155], [200, 184,...
1       [[[193, 179, 152], [203, 192, 164], [216, 204,...
2       [[[114, 87, 10], [121, 94, 17], [113, 87, 10],...
3       [[[193, 179, 152], [196, 182, 155], [204, 190,...
4       [[[147, 127, 118], [154, 136, 122], [156, 140,...
                              ...                        
4495    [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...
4496    [[[133, 142, 156], [141, 150, 164], [138, 149,...
4497    [[[77, 82, 121], [64, 69, 108], [56, 63, 102],...
4498    [[[247, 249, 249], [246, 248, 248], [245, 247,...
4499    [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...
Length: 4500, dtype: object

In [59]:
Image.fromarray(flipped_images[14]).show()

In [69]:
fake_images_list = []
dir = 'images/photo_jpg'
for image in os.listdir(dir)[0:4800]:
    fake_images_list.append(np.asarray(Image.open(dir +'/' + image)))
fake_images = pd.Series(fake_images_list)
fake_images

0       [[[113, 110, 101], [113, 110, 101], [112, 112,...
1       [[[0, 60, 91], [2, 73, 103], [0, 64, 96], [4, ...
2       [[[53, 59, 45], [55, 61, 47], [58, 65, 49], [6...
3       [[[79, 86, 96], [80, 87, 97], [82, 89, 99], [8...
4       [[[131, 173, 195], [130, 172, 194], [129, 171,...
                              ...                        
4795    [[[144, 124, 123], [144, 124, 123], [146, 125,...
4796    [[[49, 62, 70], [53, 66, 74], [56, 69, 77], [5...
4797    [[[58, 86, 133], [55, 86, 133], [53, 84, 131],...
4798    [[[149, 190, 234], [152, 193, 237], [152, 193,...
4799    [[[30, 40, 29], [29, 39, 28], [30, 40, 29], [3...
Length: 4800, dtype: object

In [70]:
Image.fromarray(fake_images_list[0]).show()

In [99]:
import torch
from torchvision.transforms.v2 import ToTensor, Normalize, Compose, ToDtype

In [113]:
transforms = Compose([ToDtype(torch.float32, scale=True), Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])])

In [154]:
reshape = lambda image : image.reshape(3, 256, 256,)
train_data = transforms(torch.tensor(pd.concat([images.apply(reshape), flipped_images.apply(reshape)], ignore_index = True)))

In [151]:
joined = pd.concat([images.apply(reshape), flipped_images.apply(reshape)], ignore_index = True)

In [153]:
joined[0].shape

(3, 256, 256)

In [183]:
train_data.shape

torch.Size([4800, 3, 256, 256])

In [157]:
fake_images = transforms(torch.tensor(fake_images.apply(reshape)))

In [184]:
fake_images.shape

torch.Size([4800, 3, 256, 256])

In [162]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size = 24, shuffle = True)

In [163]:
fake_images_dataloader = torch.utils.data.DataLoader(fake_images, batch_size = 24, shuffle = True)

In [194]:
latent_dim = 256
lr = 0.0002
beta1 = 0.5
beta2 = 0.999
num_epochs = 10

In [166]:
import torch.nn as nn

In [167]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32, momentum=0.78),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img

In [168]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.25),
        nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
        nn.ZeroPad2d((0, 1, 0, 1)),
        nn.BatchNorm2d(64, momentum=0.82),
        nn.LeakyReLU(0.25),
        nn.Dropout(0.25),
        nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
        nn.BatchNorm2d(128, momentum=0.82),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.25),
        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(256, momentum=0.8),
        nn.LeakyReLU(0.25),
        nn.Dropout(0.25),
        nn.Flatten(),
        nn.Linear(256 * 5 * 5, 1),
        nn.Sigmoid()
    )

    def forward(self, img):
        validity = self.model(img)
        return validity

In [180]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [195]:
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)
adversarial_loss = nn.BCELoss()

optimizer_G = torch.optim.Adam(generator.parameters()\
                         , lr=lr, betas=(beta1, beta2))
optimizer_D = torch.optim.Adam(discriminator.parameters()\
                         , lr=lr, betas=(beta1, beta2))

In [196]:
for epoch in range(num_epochs):
    for i, batch in enumerate(train_dataloader):
        real_paintings = batch[0].to(device)
        print(batch.shape)
        print(real_paintings.shape)

        valid = torch.ones(real_paintings.size(0), 1, device=device)
        fake = torch.zeros(real_paintings.size(0), 1, device=device)

        real_paintings = real_paintings.to(device)

        optimizer_D.zero_grad()

        z = fake_images[i*24]

        fake_paintings = generator(z)

        real_loss = adversarial_loss(discriminator\
                                     (real_paintings), valid)
        fake_loss = adversarial_loss(discriminator\
                                     (fake_paintings.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        gen_images = generator(z)

        g_loss = adversarial_loss(discriminator(gen_images), valid)

        g_loss.backward()
        optimizer_G.step()

        if (i + 1) % 100 == 0:
            print(
                f"Epoch [{epoch+1}/{num_epochs}]\
                        Batch {i+1}/{len(train_dataloader)} "
                f"Discriminator Loss: {d_loss.item():.4f} "
                f"Generator Loss: {g_loss.item():.4f}"
            )
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            z = torch.randn(16, latent_dim, device=device)
            generated = generator(z).detach().cpu()
            grid = torchvision.utils.make_grid(generated,\
                                        nrow=4, normalize=True)
            plt.imshow(np.transpose(grid, (1, 2, 0)))
            plt.axis("off")
            plt.show()

torch.Size([24, 3, 256, 256])
torch.Size([3, 256, 256])


RuntimeError: unflatten: Provided sizes [128, 8, 8] don't multiply up to the size of dim 1 (256) in the input tensor